In [23]:
import pandas as pd
from tqdm import tqdm
import numpy as np

In [7]:
df = pd.read_csv('../metadata/predictions_0.1_0.3_avg.csv')

## Splitting Objects

In [8]:
df['PredictionString'] = df['PredictionString'].str.split('/m')
df = (df
 .set_index(['ImageId'])['PredictionString']
 .apply(pd.Series)
 .stack()
 .reset_index()
 .rename(columns={0:'PredictionString'}))

## Deleting NaN values

In [9]:
df['PredictionString'].replace('', np.nan, inplace=True)
df.dropna(subset=['PredictionString'], inplace=True)

In [10]:
df['PredictionString'] = '/m' + df['PredictionString'].astype(str)

## Deviding it to 6 columns

In [11]:
df[['LabelName','Score', 'XMin', 'YMin', 'XMax', 'YMax', 'space']] = df['PredictionString'].str.split(' ', expand=True)

In [12]:
df = df[['ImageId', 'LabelName', 'Score', 'XMin', 'YMin', 'XMax', 'YMax']]

## Deleting objects under 0.31 Score

In [5]:
df = df[df['Score'].astype('float') >= 0.31]

## Replacing LabelName

In [10]:
desc = pd.read_csv('../metadata/class-descriptions-boxable.csv')

In [11]:
desc_labels = dict(zip(desc.LabelID, desc.LabelName))
df['LabelName'] = df.LabelName.map(desc_labels)

In [6]:
df.to_csv('../metadata/predictions_cleaned_v3.csv', index=False)